# Parameter definition
Choose all the settings for the database creation.

In [1]:
#TODO: Change absolute to relative paths for input and output dir.
db_name= 'test_video_dataset' #'colab_main_dataset'
input_dir = '/home/andrea/Desktop/image_datasets'
output_dir = '/home/andrea/Desktop/sql_database'
database_names=['lfw'] # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
detector_names=['mtcnn'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=['gender', 'age']  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser-fiq'] # 'confusion_score',
save_in_drive = False # To work in google colab

# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [2]:
from sql_face.sqldb import SQLDataBase

In [3]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

Creating Db file at /home/andrea/Desktop/sql_database/test_video_dataset.db


In [4]:
database.__dict__

{'db_name': 'test_video_dataset',
 'input_dir': '/home/andrea/Desktop/image_datasets',
 'save_in_drive': False,
 'output_dir': '/home/andrea/Desktop/sql_database',
 'session': <sqlalchemy.orm.session.Session at 0x7f9d7331dc50>,
 'databases': [<sql_face.databases.LFW at 0x7f9ce15dce90>],
 'detector_names': ['mtcnn'],
 'embedding_model_names': ['ArcFace'],
 'quality_model_names': ['ser-fiq']}

In [5]:
database.create_tables()

Creating image record from LFW: 13233it [00:00, 32704.30it/s]
TRIM Creating Cropped Images for detector mtcnn:   0%|          | 0/5 [00:00<?, ?it/s]

1/1 [==============================] - 0s 219ms/step


TRIM Creating Cropped Images for detector mtcnn:  20%|██        | 1/5 [00:02<00:09,  2.35s/it]

1/1 [==============================] - 0s 88ms/step


TRIM Creating Cropped Images for detector mtcnn:  40%|████      | 2/5 [00:03<00:05,  1.88s/it]

...

TRIM Creating Cropped Images for detector mtcnn:  60%|██████    | 3/5 [00:05<00:03,  1.81s/it]

1/1 [==============================] - 0s 80ms/step


TRIM Creating Cropped Images for detector mtcnn:  80%|████████  | 4/5 [00:07<00:01,  1.66s/it]

1/1 [==============================] - 0s 80ms/step


Quality models: 100%|██████████| 1/1 [00:00<00:00, 36.81it/s]


In [6]:
database.update_tables(attributes_to_update)

Updating attributes in databases:   0%|          | 0/1 [00:00<?, ?it/s]

Database <sql_face.databases.LFW object at 0x7f9ce15dce90>


1/1 [==============================] - 0s 471ms/step


1/1 [==============================] - 0s 378ms/step


1/1 [==============================] - 0s 500ms/step


...

1/1 [==============================] - 0s 435ms/step


1/1 [==============================] - 0s 465ms/step


1/1 [==============================] - 1s 582ms/step


Updating attributes in databases:   0%|          | 0/1 [02:33<?, ?it/s]


KeyboardInterrupt: 